In [ ]:
import torch
import numpy as np
import random
from torchvision import transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as datasets
import torch.nn as nn
from torch.utils.data import Subset

# Set random seeds to ensure reproducibility
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# Gaussian noise function
def apply_gaussian_noise(image, noise_level=3.0):
    """Add Gaussian noise to a single image"""
    image = image.clone()  # Copy the image to avoid modifying the original data
    noise = torch.randn(image.shape) * noise_level  # Generate Gaussian noise
    noisy_image = torch.clamp(image + noise, 0, 1)  # Limit the range of pixel values
    return noisy_image

# data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Standardized
])

# Load the full MNIST dataset
full_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

from sklearn.model_selection import train_test_split

# Get all the labels
labels = [y for _, y in full_trainset]

# Hierarchical sampling
_, subset_indices = train_test_split(
    range(len(full_trainset)),
    test_size=1/2,
    stratify=labels,
    random_state=42
)

trainset = Subset(full_trainset, subset_indices)

# Check the category distribution
from collections import Counter
print("category distribution:", Counter([full_trainset[i][1] for i in subset_indices]))

# Generate an index of the attacked samples
num_train_attack = len(trainset) // 2  # 50% of the training set is attacked to 20% attacked
# num_test_attack = len(testset) // 2  # 50% of test sets are attacked

attack_train_indices = set(random.sample(range(len(trainset)), num_train_attack))  # Randomly selected samples to add noise
train_clean_indices = set(range(len(trainset))) - attack_train_indices  # The other half stays clean

#attack_test_indices = set(random.sample(range(len(testset)), num_test_attack))  # Randomly selected samples to add noise
#test_clean_indices = set(range(len(testset))) - attack_test_indices  # The other half stays clean

# Custom datasets
class NoisyMNISTDataset(Dataset):
    def __init__(self, dataset, attacked_indices, noise_level=0.1):
        self.dataset = dataset
        self.attacked_indices = attacked_indices
        self.noise_level = noise_level

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        if idx in self.attacked_indices:
            img = apply_gaussian_noise(img, self.noise_level)  # 添加高斯噪声 
        return img, label

# Generate a training set after the attack
trainset_noisy = NoisyMNISTDataset(trainset, attack_train_indices, noise_level=5.0)  # Set the noise intensity
# testset_noisy = NoisyMNISTDataset(testset, attack_test_indices, noise_level=5.0)

# Create a DataLoader
trainloader = DataLoader(trainset_noisy, batch_size=64, shuffle=False)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

# Print the number of attacked and unattacked samples
print(f"Train Clean Samples: {len(train_clean_indices)}, Attacked Samples: {len(attack_train_indices)}")
# print(f"Test Clean Samples: {len(test_clean_indices)}, Attacked Samples: {len(attack_test_indices)}")

# Visualize partial samples
def show_images(dataset, indices, title):
    fig, axes = plt.subplots(1, 5, figsize=(10, 2))
    for ax, idx in zip(axes, list(indices)[:5]):  # 5 randomly selected
        img, label = dataset[idx]
        ax.imshow(img.squeeze(), cmap='gray')
        ax.set_title(f"Label: {label}")
        ax.axis('off')
    plt.suptitle(title)
    plt.show()

# Shows partially attacked and unattacked samples
show_images(trainset_noisy, attack_train_indices, "Attacked Train Samples")
show_images(trainset_noisy, train_clean_indices, "Clean Train Samples")

# Generate the training set and test set after the attack (the original code remains unchanged)
trainset_noisy = NoisyMNISTDataset(trainset, attack_train_indices, noise_level=5.0)
# testset_noisy = NoisyMNISTDataset(testset, attack_test_indices, noise_level=5.0)

# Split the training set into attacked and clean subsets
train_attack_subset = Subset(trainset_noisy, list(attack_train_indices))  # Attacked training samples
train_clean_subset = Subset(trainset_noisy, list(train_clean_indices))    # Clean training samples

# test_attack_subset = Subset(testset_noisy, list(attack_test_indices))     # Attacked testing samples
# test_clean_subset = Subset(testset_noisy, list(test_clean_indices))       # Clean testing samples

# Create the corresponding DataLoader
attackTrainloader = DataLoader(train_attack_subset, batch_size=64, shuffle=False)
cleanTrainloader = DataLoader(train_clean_subset, batch_size=64, shuffle=False)


# Print each subset size
print(f"Train - Attack: {len(train_attack_subset)}, Clean: {len(train_clean_subset)}")
# print(f"Test - Attack: {len(test_attack_subset)}, Clean: {len(test_clean_subset)}")


In [ ]:
# Adding USPS dataset
import numpy as np
from sklearn.datasets import fetch_openml
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch
from PIL import Image

# Custom USPS dataset class
class USPSTestSet(Dataset):
    def __init__(self, transform=None):
        # Load complete USPS data (9298 samples)
        usps = fetch_openml('usps', version=1, parser='auto')
        self.data = usps['data'].values.reshape(-1, 16, 16).astype(np.float32)
        self.labels = usps['target'].astype(np.int64)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = Image.fromarray(self.data[idx], mode='F')  # Mode 'F' for floating-point image
        label = self.labels[idx]
        
        if self.transform:
            img = self.transform(img)
            
        return img, label

# Define transforms compatible with MNIST
transform = transforms.Compose([
    transforms.Resize(28),                  # USPS 16x16 -> MNIST 28x28
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Using MNIST normalization parameters
])

# Create complete USPS test set
usps_testset = USPSTestSet(transform=transform)
usps_loader = DataLoader(usps_testset, batch_size=64, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
import time

# Define FNN network
class FNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = x.view(-1, 784)  # Flatten input
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Initialize model, loss function and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train model (with time tracking)
epochs = 5
total_train_time = 0  # Track total training time

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    epoch_start_time = time.time()  # Record epoch start time
    
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    epoch_time = time.time() - epoch_start_time  # Calculate epoch duration
    total_train_time += epoch_time
    
    print(f"Epoch {epoch + 1}/{epochs}, "
          f"Loss: {running_loss / len(trainloader):.4f}, "
          f"Time: {epoch_time:.2f}s")

# Output total training time
print(f"\nTotal training time: {total_train_time:.2f} seconds")
print(f"Average time per epoch: {total_train_time/epochs:.2f}s")
time_source  = total_train_time/epochs

In [ ]:
# Model performance on test set
# Validate the model
model.eval()  # Set model to evaluation mode
correct = 0
total = 0
# Ensure input data is on the same device during validation
with torch.no_grad():  # Disable gradient calculation
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to device
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)  # Get predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Print accuracy
print(f"Validation Accuracy: {100 * correct / total:.2f}%")


In [ ]:
## Attribution Calculation - Considering the last layer 128->64->10, computing for the 64-channel layer

# Calculate FFN layer attribution
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Automatically select device
model.to(device)  # Move model to GPU

In [ ]:
def compute_ffn_attribution(model, data_loader):
    model.eval()
    attributions = []
    leaky_relu = torch.nn.LeakyReLU(0.1)  # Define activation function
    
    for images, labels in data_loader:
        images = images.to(device)
        images.requires_grad_()

        # Forward passing
        # x = images.view(-1, 28*28)
        # x = leaky_relu(model.fc1(x))  # Using LeakyReLU
        # fc2_outputs = leaky_relu(model.fc2(x))
        # y = model.fc3(fc2_outputs)
        # Forward passing (using ReLU)
        x = images.view(-1, 28*28)
        x = F.relu(model.fc1(x))  # Changed to ReLU
        fc2_outputs = F.relu(model.fc2(x))  # Changed to ReLU
        y = model.fc3(fc2_outputs)
        
        # Compute gradients
        gradient_matrix = []
        for i in range(10):  # Compute gradients for each output dimension
            model.zero_grad()
            grad_i = torch.autograd.grad(
                outputs=y[:, i],
                inputs=fc2_outputs,
                grad_outputs=torch.ones_like(y[:, i]),
                retain_graph=True,
                create_graph=False
            )[0]  # (batch_size, 64)
            gradient_matrix.append(grad_i)
        
        # Stack and aggregate gradients
        gradient_matrix = torch.stack(gradient_matrix, dim=1)  # (batch_size, 10, 64)
        
        # Improved aggregation methods - pure mean and norm are incorrect approaches
        
        # Method 3: Gradient-weighted by activation values
        attribution_weighted = (gradient_matrix.mean(dim=1) * fc2_outputs).abs()
        # Max aggregation
        attribution_max = (gradient_matrix.max(dim=1)[0] * fc2_outputs).abs()  # (batch, 64)
        # Here we select Method 3 as example
        
        # Top-k clustering
        # k = 3
        # topk_grads = gradient_matrix.topk(k, dim=1)[0]  # (batch, k, 64)
        # attribution = (topk_grads.mean(dim=1) * fc2_outputs).abs()
        attributions.append(attribution_max.detach().cpu())
    
    return torch.cat(attributions, dim=0)  # (num_samples, 64)

In [ ]:
# Make sure the training data is also on the GPU
start_time = time.time()
ffn_attributions = compute_ffn_attribution(model, trainloader)
end_time = time.time()-start_time
print(f"\ncal attributions cost time: {end_time:.2f} seconds")
ffn_attributions.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

# Assume ffn_attributions is a PyTorch tensor on GPU
# 1. Move tensor from GPU to CPU
ffn_attributions_cpu = ffn_attributions.cpu()

# 2. Convert PyTorch tensor to NumPy array
ffn_attributions_np = ffn_attributions_cpu.numpy()

# 3. Standardize using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(ffn_attributions_np)

# Print standardized data
# print("Scaled data:")
# print(X_scaled)

In [ ]:
from sklearn.cluster import KMeans

# Perform clustering using K-Means
kmeans = KMeans(n_clusters=2, random_state=42)  # Set K=2
kmeans.fit(X_scaled)

# Get cluster labels
labels = kmeans.labels_

# Get cluster centers
centers = kmeans.cluster_centers_

# Cluster analysis
import numpy as np

# Assume labels is the label array after K-Means clustering
# labels is a 1D array with shape (n_samples,)

# Get sample indices belonging to cluster 0
cluster_0_indices = np.where(labels == 0)[0]
print(cluster_0_indices.size)
# Get sample indices belonging to cluster 1
cluster_1_indices = np.where(labels == 1)[0]
print(cluster_1_indices.size)

# Print results
print("Indices of samples in cluster 0:", cluster_0_indices)
print("Indices of samples in cluster 1:", cluster_1_indices)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import numpy as np

# PCA for dimensionality reduction visualization (optional)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# K-Means clustering
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
kmeans_labels = kmeans.fit_predict(X_scaled)

# Get indices for both clusters
cluster_0_indices = np.where(kmeans_labels == 0)[0]
cluster_1_indices = np.where(kmeans_labels == 1)[0]

print(f"K-Means clustering results:")
print(f"Cluster 0 sample count: {len(cluster_0_indices)}")
print(f"Cluster 1 sample count: {len(cluster_1_indices)}")

# Visualization (optional)
import matplotlib.pyplot as plt
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=kmeans_labels, alpha=0.5)
plt.title('K-Means Clustering Results')
plt.show()

In [ ]:
from sklearn.mixture import GaussianMixture

# GMM clustering
gmm = GaussianMixture(n_components=2, random_state=42)
gmm_labels = gmm.fit_predict(X_scaled)

# Get indices for both clusters
cluster_0_indices = np.where(gmm_labels == 0)[0]
cluster_1_indices = np.where(gmm_labels == 1)[0]

print(f"\nGMM clustering results:")
print(f"Cluster 0 sample count: {len(cluster_0_indices)}")
print(f"Cluster 1 sample count: {len(cluster_1_indices)}")

In [ ]:
# Calculate the matching between K-Means clustering and manual partitioning
attack_set = set(attack_train_indices)  # True attack sample indices
clean_set = set(train_clean_indices)    # True clean sample indices

cluster_0_set = set(cluster_0_indices)  # Samples in K-Means cluster 0
cluster_1_set = set(cluster_1_indices)  # Samples in K-Means cluster 1

# Calculate intersections
attack_in_cluster_0 = len(attack_set & cluster_0_set)  # Number of true attack samples in Cluster 0
attack_in_cluster_1 = len(attack_set & cluster_1_set)  # Number of true attack samples in Cluster 1

clean_in_cluster_0 = len(clean_set & cluster_0_set)  # Number of true clean samples in Cluster 0
clean_in_cluster_1 = len(clean_set & cluster_1_set)  # Number of true clean samples in Cluster 1

# Calculate classification accuracy of K-Means for attack samples
attack_accuracy_cluster_0 = (attack_in_cluster_0 / len(attack_set)) * 100
attack_accuracy_cluster_1 = (attack_in_cluster_1 / len(attack_set)) * 100

clean_accuracy_cluster_0 = (clean_in_cluster_0 / len(clean_set)) * 100
clean_accuracy_cluster_1 = (clean_in_cluster_1 / len(clean_set)) * 100

# Print comparison results
print("==== K-Means Clustering vs Manual Labels ====")
print(f"Cluster 0: {len(cluster_0_indices)} samples")
print(f"Cluster 1: {len(cluster_1_indices)} samples\n")

print(f"Attack Samples in Cluster 0: {attack_in_cluster_0} ({attack_accuracy_cluster_0:.2f}%)")
print(f"Attack Samples in Cluster 1: {attack_in_cluster_1} ({attack_accuracy_cluster_1:.2f}%)\n")

print(f"Clean Samples in Cluster 0: {clean_in_cluster_0} ({clean_accuracy_cluster_0:.2f}%)")
print(f"Clean Samples in Cluster 1: {clean_in_cluster_1} ({clean_accuracy_cluster_1:.2f}%)")

# Calculate overall classification accuracy of K-Means
total_correct = attack_in_cluster_0 + clean_in_cluster_1  # Assuming cluster_0 mainly contains attack samples, cluster_1 mainly clean samples
overall_accuracy = (total_correct / len(trainset)) * 100  # Previously train_subset, now changed to trainset

print(f"\nOverall Clustering Accuracy: {overall_accuracy:.2f}%")

In [ ]:
# Extract samples from cluster 0
cluster_0_attributions = ffn_attributions[cluster_0_indices]

# Extract samples from cluster 1
cluster_1_attributions = ffn_attributions[cluster_1_indices]
print(cluster_0_attributions.shape)
print(cluster_1_attributions.shape) 


In [ ]:
import torch
from torch.utils.data import DataLoader, Subset

# Ensure labels length matches dataset size gam_labels
# assert len(labels) == len(trainloader.dataset), "labels and trainloader.dataset size mismatch!"

# Convert indices to Python list
cluster_0_indices = cluster_0_indices.tolist()
cluster_1_indices = cluster_1_indices.tolist()

# Use Subset to split original dataset by indices
dataset_0 = Subset(trainloader.dataset, cluster_0_indices)
dataset_1 = Subset(trainloader.dataset, cluster_1_indices)

# Create new DataLoaders
trainloader_0 = DataLoader(dataset_0, batch_size=64, shuffle=True) #
trainloader_1 = DataLoader(dataset_1, batch_size=64, shuffle=True)

# Final verification
print(f"Expected Cluster 0 size: {len(cluster_0_indices)}, Actual: {len(trainloader_0.dataset)}")
print(f"Expected Cluster 1 size: {len(cluster_1_indices)}, Actual: {len(trainloader_1.dataset)}")

# Assign datasets based on clustering accuracy
if overall_accuracy > 0.5:
    fine_tuning_load = trainloader_1
else:
    fine_tuning_load = trainloader_0

In [ ]:
#模型在测试集上的性能
# 验证模型
model.eval()  # Set model to evaluation mode
correct = 0
total = 0
# Ensure input data is on the same device during validation
with torch.no_grad():  # Disable gradient calculation
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to device
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)  # Get predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Print accuracy
print(f"Validation Accuracy: {100 * correct / total:.2f}%")

In [ ]:
import torch

def evaluate_model(model, dataloader, device):
    """
    Calculate model accuracy on specified dataset.

    Parameters:
    - model: Trained PyTorch model
    - dataloader: DataLoader for evaluation
    - device: Computation device ('cuda' or 'cpu')

    Returns:
    - accuracy: Accuracy score (float)
    """
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation for faster inference
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to device
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)  # Get predictions
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [ ]:
# Evaluate model on cluster_0 and cluster_1 datasets
accuracy_cluster_0 = evaluate_model(model, trainloader_0, device)
accuracy_cluster_1 = evaluate_model(model, trainloader_1, device)
accuracy_cluster = evaluate_model(model,trainloader,device)

print(f"Accuracy on Cluster 0: {accuracy_cluster_0:.2f}%")
print(f"Accuracy on Cluster 1: {accuracy_cluster_1:.2f}%")
print(f"Accuracy on Cluster : {accuracy_cluster:.2f}%")

accuracy_attack_train = evaluate_model(model, attackTrainloader , device)
accuracy_clean_train = evaluate_model(model, cleanTrainloader , device)
# accuracy_attack_test = evaluate_model(model, attackTestloader , device)
accuracy_test = evaluate_model(model, testloader , device)

print(f"Accuracy on attack_train : {accuracy_attack_train:.2f}%")
print(f"Accuracy on clean_train : {accuracy_clean_train:.2f}%")

print(f"Accuracy on test : {accuracy_test:.2f}%")
accuracy_usps = evaluate_model(model, usps_loader , device)
print(f"Accuracy on usps_test : {accuracy_usps:.2f}%")
#print(f"Accuracy on clean_test : {accuracy_clean_test:.2f}%")
accuracy_source = accuracy_test

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def contrastive_loss(anchor, samples, temperature=0.1):
    """
    Compute contrastive loss (InfoNCE Loss)
    :param anchor: Anchor vector (shape: [64])
    :param samples: Sample matrix (shape: [n_samples, 64])
    :param temperature: Temperature parameter
    :return: Contrastive loss
    """
    # Calculate cosine similarity between anchor and all samples
    sim_matrix = F.cosine_similarity(anchor.unsqueeze(0), samples, dim=1) / temperature
    # Contrastive loss calculation
    loss = -torch.log(torch.exp(sim_matrix[0]) / torch.sum(torch.exp(sim_matrix)))
    return loss

def find_central_sample(cluster_attributions, batch_size=1024):
    cluster_center = torch.mean(cluster_attributions, dim=0)
    min_loss = float('inf')
    best_sample = None

    # Process in batches
    for i in range(0, len(cluster_attributions), batch_size):
        batch = cluster_attributions[i:i + batch_size]
        losses = torch.stack([contrastive_loss(cluster_center, sample.unsqueeze(0)) for sample in batch])
        min_batch_loss, min_batch_idx = torch.min(losses, dim=0)
        if min_batch_loss < min_loss:
            min_loss = min_batch_loss
            best_sample = batch[min_batch_idx]

    return best_sample

# Example usage
cluster_0_central_sample = find_central_sample(cluster_0_attributions)
cluster_1_central_sample = find_central_sample(cluster_1_attributions)

print("Cluster 0 的最中心样本:", cluster_0_central_sample)
print("Cluster 1 的最中心样本:", cluster_1_central_sample)#也有好多02

In [ ]:
import copy

# Create a deep copy of the model
model_copy = copy.deepcopy(model)

# Ensure the new model is on the same device as the original
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_copy.to(device)

# Verify successful copying
print(model_copy)

In [ ]:
def compute_layer_features(model, data_loader, cluster_labels, layer_name='fc2'):
    """
    Unified computation of model layer features and corresponding cluster labels

    Args:
        model: Neural network model
        data_loader: Data loader (must return sample indices)
        cluster_labels: Cluster label array (0/1)
        layer_name: Target layer name ('fc1', 'fc2', or 'fc3')

    Returns:
        X: Feature matrix of specified layer (n_samples, feature_dim)
        y: Corresponding cluster labels (n_samples,)
    """
    model.eval()
    X_list = []
    y_list = []
    
    # Ensure labels are numpy array
    if isinstance(cluster_labels, torch.Tensor):
        cluster_labels = cluster_labels.cpu().numpy()
    
    for images, indices in data_loader:
        images = images.to(device)
        
        # Forward pass to specified layer
        with torch.no_grad():
            x = images.view(-1, 28*28)
            
            if layer_name == 'fc1':
                features = torch.relu(model.fc1(x))
            elif layer_name == 'fc2':
                x = torch.relu(model.fc1(x))
                features = torch.relu(model.fc2(x))
            elif layer_name == 'fc3':
                features = model(images)  # Full model output
            else:
                raise ValueError(f"Unsupported layer name: {layer_name}")
        
        # Store features and labels
        X_list.append(features.cpu().numpy())
        
        # Get labels for current batch (handles indices automatically)
        batch_labels = cluster_labels[indices.numpy()]
        if overall_accuracy > 0.5:
            inverse_labels =1 - batch_labels 
        else:
            inverse_labels = batch_labels
        y_list.append(inverse_labels)  # Label inversion
    
    # Concatenate results
    X = np.concatenate(X_list, axis=0)
    y = np.concatenate(y_list, axis=0)
    
    return X, y

In [ ]:
# run func
X_1,y_1 = compute_layer_features(model_copy, trainloader,gmm_labels,"fc1")
X_2,y_2 = compute_layer_features(model_copy, trainloader,gmm_labels,"fc2")
X_3,y_3 = compute_layer_features(model_copy, trainloader,gmm_labels,"fc3")

In [ ]:
def solve_linear_regression_torch(X, y):
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

    # Add bias term
    ones = torch.ones(X.shape[0], 1)
    X_bias = torch.cat([X, ones], dim=1)  # Shape becomes (60000, 65)

    # Solve using least squares
    W = torch.linalg.lstsq(X_bias, y).solution
    return W[:-1], W[-1]  # W[:-1] are weights (n_features,), W[-1] is bias
W2_torch, b2_torch = solve_linear_regression_torch(X_2, y_2)
W1_torch, b1_torch = solve_linear_regression_torch(X_1, y_1)
W3_torch, b3_torch = solve_linear_regression_torch(X_3, y_3)

print("W shape:", W2_torch.shape)  # (64,)
print("b:", b2_torch)
# Print all weights
for i, w_i in enumerate(W2_torch):
    print(f"W[{i}] = {w_i}")

In [ ]:
# Flatten W_torch into a one-dimensional tensor
W_torch = W2_torch.flatten()  # shape is (64,)

# Sort in descending order by absolute value
W_abs_sorted, indices = torch.sort(torch.abs(W_torch), descending=True)

# Take the indices of the top 10 largest neurons
top_10_indices = indices[:20]
top_10_values = W_torch[top_10_indices]  # Get the corresponding W values

# Print the contents of top_10_values
print("Top 10 values:", top_10_values)

# output
print("Top 10 neurons with highest absolute weights:")
for rank, (idx, val) in enumerate(zip(top_10_indices.tolist(), top_10_values.tolist()), start=1):
    print(f"Rank {rank}: Neuron {idx}, Weight = {float(val):.6f}")  # Ensure val is a float How to prune
# Get the weights and biases of the fc2 layer
fc2_weight = model_copy.fc2.weight  # Weight matrix, shape (out_features, in_features)
fc2_bias = model_copy.fc2.bias      # Bias vector, shape (out_features,)

# Assume top_10_indices are the indices of the top 10 neurons obtained earlier
#top_10_indices = torch.tensor([12, 45, 3, 28, 7, 19, 33, 56, 22, 41])  # Example data
neurons_to_zero = top_10_indices[:20]  # Take the first 7 neurons
print("Neurons to zero:", neurons_to_zero)

# Gradually set the weights and biases to 0
for neuron_idx in neurons_to_zero:
    # Set the corresponding neuron's weights to 0
    fc2_weight.data[neuron_idx, :] = 0  # Set the weight row of this neuron to 0

    # Set the corresponding neuron's bias to 0
    #fc2_bias.data[neuron_idx] = 0  # Set the weight row of this neuron to 0

    print(f"Neuron {neuron_idx} weight and bias set to 0.")

# Print the modified weights and biases
print("Modified fc2 weight:")
print(fc2_weight)

print("Modified fc2 bias:")
print(fc2_bias)

In [ ]:
# Modify fc1 according to fc2 with 40 neurons
# Flatten W_torch into a one-dimensional tensor
W_torch = W1_torch.flatten()  # shape is (128,)

# Sort in descending order by absolute value
W_abs_sorted, indices = torch.sort(torch.abs(W_torch), descending=True)

# Take the indices of the top 10 largest neurons
top_10_indices = indices[:40]  # Take the first 10 indices 40 / 128
top_10_values = W_torch[top_10_indices]  # Get the corresponding W values

# Print the contents of top_10_values
print("Top 10 values:", top_10_values)

# output
print("Top 10 neurons with highest absolute weights:")
for rank, (idx, val) in enumerate(zip(top_10_indices.tolist(), top_10_values.tolist()), start=1):
    print(f"Rank {rank}: Neuron {idx}, Weight = {float(val):.6f}")  # Ensure val is a float How to prune
# Get the weights and biases of the fc2 layer
fc1_weight = model_copy.fc.weight  # Weight matrix, shape (out_features, in_features)
print(fc1_weight.shape)
fc1_bias = model_copy.fc.bias      # Bias vector, shape (out_features,)

# Assume top_10_indices are the indices of the top 10 neurons obtained earlier
#top_10_indices = torch.tensor([12, 45, 3, 28, 7, 19, 33, 56, 22, 41])  # Example data
neurons_to_zero = top_10_indices[:40]  # Take the first 7 neurons
print("Neurons to zero:", neurons_to_zero)

# Gradually set the weights and biases to 0
for neuron_idx in neurons_to_zero:
    # Set the corresponding neuron's weights to 0
    fc1_weight.data[neuron_idx, :] = 0  # Set the weight row of this neuron to 0

    # Set the corresponding neuron's weights to 0
    # fc2_bias.data[neuron_idx] = 0  # Set the weight row of this neuron to 0

    print(f"Neuron {neuron_idx} weight and bias set to 0.")

In [ ]:
# Ensure the data is on the GPU (if available)
# Test on the pruned model_copy
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_copy.to(device)

# Evaluate the model on the cluster_0 and cluster_1 datasets
accuracy_cluster_0 = evaluate_model(model_copy, trainloader_0, device)
accuracy_cluster_1 = evaluate_model(model_copy, trainloader_1, device)
accuracy_cluster = evaluate_model(model_copy,trainloader,device)

print(f"Accuracy on Cluster 0: {accuracy_cluster_0:.2f}%")
print(f"Accuracy on Cluster 1: {accuracy_cluster_1:.2f}%")
print(f"Accuracy on Cluster : {accuracy_cluster:.2f}%")

accuracy_attack_train = evaluate_model(model_copy, attackTrainloader , device)
accuracy_clean_train = evaluate_model(model_copy, cleanTrainloader , device)
# accuracy_attack_test = evaluate_model(model, attackTestloader , device)
accuracy_test = evaluate_model(model_copy, testloader , device)

print(f"Accuracy on attack_train : {accuracy_attack_train:.2f}%")
print(f"Accuracy on clean_train : {accuracy_clean_train:.2f}%")
accuracy_prune = accuracy_test

print(f"Accuracy on test : {accuracy_test:.2f}%")
accuracy_usps = evaluate_model(model_copy, usps_loader , device)
print(f"Accuracy on usps_test : {accuracy_usps:.2f}%")



In [ ]:
import copy

# Deep copy the model
model_copy_fine_all = copy.deepcopy(model_copy)

# Ensure the new model is on the same device as the original
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_copy_fine_all.to(device)

# Verify if the copy was successful
print(model_copy_fine_all)

In [ ]:
# Assume device is GPU or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to device
model_copy = model_copy.to(device)

# Freeze all layers except fc2
for name, param in model_copy.named_parameters():
    if not name.startswith("fc2") | name.startswith("fc1"):
        param.requires_grad = False

# Check which layers' parameters are frozen
for name, param in model_copy.named_parameters():
    print(f"{name}: requires_grad = {param.requires_grad}")

# Define optimizer
optimizer = optim.Adam(model_copy.parameters(), lr=0.001)

# Define loss function
criterion = nn.CrossEntropyLoss()

import time

# Training loop (with time statistics)
num_epochs = 5
total_training_time = 0  # Track total training time

for epoch in range(num_epochs):
    epoch_start_time = time.time()  # Record epoch start time
    model_copy.train()  # Set model to training mode
    running_loss = 0.0
    batch_times = []  # Track time per batch

    for inputs, labels in fine_tuning_load:
        batch_start_time = time.time()  # Record batch start time

        # Move input data and labels to device
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model_copy(inputs)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update parameters
        optimizer.step()

        # Record loss
        running_loss += loss.item()

        # Record batch time
        batch_time = time.time() - batch_start_time
        batch_times.append(batch_time)

    # Calculate epoch time
    epoch_time = time.time() - epoch_start_time
    total_training_time += epoch_time

    # Calculate average batch time
    avg_batch_time = sum(batch_times)/len(batch_times) if batch_times else 0

    # Print training info (with time statistics)
    print(f"Epoch [{epoch + 1}/{num_epochs}], "
          f"Loss: {running_loss / len(trainloader):.4f}, "
          f"Epoch Time: {epoch_time:.2f}s, "
          f"Avg Batch Time: {avg_batch_time*1000:.1f}ms")

# Final time statistics
print(f"\nTraining completed!")
print(f"Total training time: {total_training_time:.2f} seconds")
print(f"Average epoch time: {total_training_time/num_epochs:.2f}s")
time_fc1_fc2 = total_training_time/num_epochs

# Validate model
model_copy.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradient calculation
    for inputs, labels in testloader:
        # Move input data and labels to device
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model_copy(inputs)

        # Get predictions
        _, predicted = torch.max(outputs.data, 1)

        # Count correct predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Print accuracy
print(f"Validation Accuracy: {100 * correct / total:.2f}%")

In [ ]:
# Assume device is GPU or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to device
model_copy_fine_all = model_copy_fine_all.to(device)

# Define optimizer
optimizer = optim.Adam(model_copy_fine_all.parameters(), lr=0.001)

# Define loss function
criterion = nn.CrossEntropyLoss()

import time

# Training loop (with time tracking)
num_epochs = 5
total_training_time = 0  # Track total training time

for epoch in range(num_epochs):
    epoch_start_time = time.time()  # Record epoch start time
    model_copy_fine_all.train()  # Set model to training mode
    running_loss = 0.0
    batch_times = []  # Track time per batch
    
    for inputs, labels in fine_tuning_load:
        batch_start_time = time.time()  # Record batch start time

        # Move inputs and labels to device
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model_copy_fine_all(inputs)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update parameters
        optimizer.step()

        # Record loss
        running_loss += loss.item()

        # Track batch time
        batch_time = time.time() - batch_start_time
        batch_times.append(batch_time)

    # Calculate epoch time
    epoch_time = time.time() - epoch_start_time
    total_training_time += epoch_time

    # Calculate average batch time
    avg_batch_time = sum(batch_times)/len(batch_times) if batch_times else 0

    # Print training info (with timing)
    print(f"Epoch [{epoch + 1}/{num_epochs}], "
          f"Loss: {running_loss / len(trainloader):.4f}, "
          f"Epoch Time: {epoch_time:.2f}s, "
          f"Avg Batch Time: {avg_batch_time*1000:.1f}ms")

# Final timing statistics
print(f"\nTraining completed!")
print(f"Total training time: {total_training_time:.2f} seconds")
print(f"Average epoch time: {total_training_time/num_epochs:.2f}s")
time_all = total_training_time/num_epochs

# Validate model
model_copy_fine_all.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradient calculation
    for inputs, labels in testloader:
        # Move inputs and labels to device
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model_copy_fine_all(inputs)

        # Get predictions
        _, predicted = torch.max(outputs.data, 1)

        # Count correct predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Print accuracy
print(f"Validation Accuracy: {100 * correct / total:.2f}%")

In [ ]:
# Ensure data is on GPU (if available) - Testing on fine-tuned model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_copy.to(device)

# Evaluate model on cluster_0 and cluster_1 datasets
accuracy_cluster_0 = evaluate_model(model_copy, trainloader_0, device)
accuracy_cluster_1 = evaluate_model(model_copy, trainloader_1, device)
accuracy_cluster = evaluate_model(model_copy,trainloader,device)

print(f"Accuracy on Cluster 0: {accuracy_cluster_0:.2f}%")
print(f"Accuracy on Cluster 1: {accuracy_cluster_1:.2f}%")
print(f"Accuracy on Cluster : {accuracy_cluster:.2f}%")

accuracy_attack_train = evaluate_model(model_copy, attackTrainloader , device)
accuracy_clean_train = evaluate_model(model_copy, cleanTrainloader , device)
# accuracy_attack_test = evaluate_model(model, attackTestloader , device)
accuracy_test = evaluate_model(model_copy, testloader , device)
accuracy_test_all = evaluate_model(model_copy_fine_all, testloader , device)
print(f"Accuracy on attack_train : {accuracy_attack_train:.2f}%")
print(f"Accuracy on clean_train : {accuracy_clean_train:.2f}%")
#print(f"Accuracy on test souse : {accuracy_test:.2f}%")

print(f"Average epoch time: {time_source:.2f}s")
print(f"Accuracy on test : {accuracy_source:.2f}%")
print(f"Accuracy on test_prune : {accuracy_prune:.2f}%")
print(f"Average epoch time on special: {time_fc1_fc2:.2f}s")
print(f"Accuracy on test_fin-tuning special : {accuracy_test:.2f}%")
print(f"Average epoch time on all: {time_all:.2f}s")
print(f"Accuracy on test_fin-tuning all : {accuracy_test_all:.2f}%")